In [ ]:
# Author : Raude Killian
# Last modified 16.07.205
# Last Check by Qiaoyue 15.07.2025

In [ ]:
import mne
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
import numpy as np
%matplotlib inline

PARTICIPANTS_FILE = r"L:\Common\Users\Qiaoyue\MEG_project\Data\participants.csv"

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

participants = pd.read_csv(PARTICIPANTS_FILE)  
participants = participants['subID'].astype(str).tolist()
sessions = ["encoding", "recall"]

# Visual components from previous steps, identified by hand.
ICA_EXCLUSIONS_ENCODING = {
    "F103" : [0,3,19],
    "F104" : [0,4,16],
    "F105" : [0,11,14],
    "F106" : [0,1,3,6],
    "F107" : [0,17],
    "F108" : [0],
    "F109" : [0,9],
    "F111" : [0,2,12],
    "F112" : [0,1],
    "F113" : [1,8,33],
    "F114" : [1,5,16],
    "F115" : [4,8,28],
    "F116" : [0,8,14],
    "F117" : [1,3,10],
    "F118" : [0,2,10,11],
    "F119" : [1,12,13],
    "F120" : [1,12,21],
    "F122" : [0,19],
    "F123" : [1,2,3],
}

ICA_EXCLUSIONS_RECALL = {
    "F103" : [0,3,35,36],
    "F104" : [1,2,28],
    "F105" : [0,10,13],
    "F106" : [0,1,3,29],
    "F107" : [1,5,17,23],
    "F108" : [0,1,7,16],
    "F109" : [0,13,21],
    "F111" : [0,5,16],
    "F112" : [0],
    "F113" : [1,10,40],
    "F114" : [1,3,21],
    "F115" : [3,4],
    "F116" : [1,8,15],
    "F117" : [1,2,3,31],
    "F118" : [0,16],
    "F119" : [2,6,7],
    "F120" : [1,7,22,34],
    "F122" : [0,29],
    "F123" : [1,2,4],
}

print(f"✅ {len(participants)} participants list : {participants}")

In [ ]:
# For tests purposes
#participants = ["F103"]
#sessions = ["encoding"]

In [ ]:
#Fully apply ICA
components = []
visual_compo = []
muscle_compo = []
ecg_compo = []
resp_compo = []
ICA_EXCLUSIONS = 0

for subID in participants:
    for session in sessions:

        if session == "encoding":
            ICA_EXCLUSIONS = ICA_EXCLUSIONS_ENCODING
        elif session == "recall":
            ICA_EXCLUSIONS = ICA_EXCLUSIONS_RECALL

        plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
        os.makedirs(plots_folder, exist_ok=True)
        
        #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
        intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
        os.makedirs(intermediates_folder, exist_ok=True)

        print(f"Preprocessing: {subID}_{session}_session")

        # Declaring filenames
        sss_meg_filename = rf"{intermediates_folder}/{subID}_{session}_sss.fif"
        sss_meg_file = mne.io.read_raw_fif(sss_meg_filename, preload=True, verbose=False)

        sss_meg_file_filtered = sss_meg_file.copy().pick('meg')
        sss_meg_file_filtered.resample(200, verbose=False) 
        sss_meg_file_filtered.filter(1, 40, verbose=False)  

        # get ECG channel
        ecg_raw = sss_meg_file.copy().pick(['ECG001'])
        ecg_raw.resample(200, verbose=False)
        sss_meg_file_filtered.add_channels([ecg_raw], force_update_info=True)

        # get RESP channel
        bio_raw = sss_meg_file.copy().pick(['BIO002'])
        bio_raw.resample(200, verbose=False)
        sss_meg_file_filtered.add_channels([bio_raw], force_update_info=True)

        # Compute ICA
        ica = mne.preprocessing.ICA(n_components=0.99, method='fastica', random_state=1906, verbose=False) 
        ica.fit(sss_meg_file_filtered, verbose=False)
        components.append(ica.n_components_)

        #Visual
        visual_compo.append(len(ICA_EXCLUSIONS.get(subID, [])))
        print(f"Bad visually inspected components {subID}: {ICA_EXCLUSIONS.get(subID, [])}")
        
        #Muscle
        muscle_indices, _ = ica.find_bads_muscle(sss_meg_file_filtered, verbose=False)
        muscle_compo.append(len(muscle_indices))
        print(f"Bad Muscle-related components {subID}: {muscle_indices}")

        #ECG
        ecg_indices, _ = ica.find_bads_ecg(sss_meg_file_filtered, ch_name='ECG001', verbose=False)
        ecg_compo.append(len(ecg_indices))
        print(f"Bad ECG-related components {subID}: {ecg_indices}")

        #Resp
        resp_indices, _ = ica.find_bads_eog(sss_meg_file_filtered, ch_name='BIO002', verbose=False) 
        resp_compo.append(len(resp_indices))
        print(f"Bad respiration-related components {subID}: {resp_indices}")

        #Exclude
        exclusions = ICA_EXCLUSIONS.get(subID, [])
        exclusions_conc = np.concatenate((exclusions,muscle_indices,ecg_indices,resp_indices))
        unique_exclusions = np.unique(exclusions_conc)
        print(f"ICA compenents to remove: {unique_exclusions}")

        # Apply ICA
        ica.exclude = unique_exclusions
        ica.apply(sss_meg_file, verbose=False)

        sss_meg_file.save(f"{intermediates_folder}/{subID}_{session}_icaed.fif", overwrite=True, verbose = False)
        print(f"✅ {subID}_{session}_session sucessfully completed  \n")

print(f"Mean number of ICA components:                  {np.mean(components)} ± {np.std(components)}")
print(f"Mean number of removed visual components:       {np.mean(visual_compo)} ± {np.std(visual_compo)}")
print(f"Mean number of removed muscle components:       {np.mean(muscle_compo)} ± {np.std(muscle_compo)}")
print(f"Mean number of removed ecg components:          {np.mean(ecg_compo)} ± {np.std(ecg_compo)}")
print(f"Mean number of removed respiration components:  {np.mean(resp_compo)} ± {np.std(resp_compo)}")

print(f"Visual components {visual_compo}")
print(f"Muscle components {muscle_compo}")
print(f"ECG components {ecg_compo}")
print(f"Respiratory components {resp_compo}")